In [1]:
import os
import numpy as np
print(f"当前目录: {os.getcwd()}")
if 'arch-gaussian' in os.listdir():
    os.chdir('arch-gaussian')
    print(f"更改后的目录: {os.getcwd()}")


当前目录: D:\M.Arch\MastersThesis\arch-gaussian\arch-gaussian\scripts
更改后的目录: D:\M.Arch\MastersThesis\arch-gaussian\arch-gaussian


In [2]:
import sys
sys.path.append('./src')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import config
config.scene_name = 'Hongkong\ShenShuiBu\ShenShuiBuComposite'
config.vid_name = 'VID.mp4' # 如果不是抽帧可以不填
config.vid_frames = 30 # 如果不是抽帧可以不填
indent_frames = 0 # 视频编号从第几帧开始
config.update_args()
config.update_colmap_args()

### STEP1 FFMPEG (从视频抽帧)

In [ ]:
from utils.video_utils import extract_frames
print("请确认以下参数:")
print(f"文件夹 = {config.args.source_path}")
print(f"要抽取的帧数 = {config.vid_frames}")
print(f"视频名称 = {config.vid_name}")
print(f"indent_frames = {indent_frames}")

In [ ]:
if not os.path.exists(os.path.join(config.args.source_path, "input")):
    success, frames_folder = extract_frames(os.path.join(config.args.source_path, config.vid_name), config.vid_frames, indent_frames)
else:
    print("[!]检测到抽帧已完成，已为您跳过")


### STEP1 从Google Earth Studio渲染的图片

In [ ]:
import shutil
from PIL import Image
from tqdm import tqdm

In [ ]:
# 指定文件夹路径
input_folder_path = os.path.join(config.args.source_path, "footage")
output_folder_path = os.path.join(config.args.source_path, "input")
step = 1
resize = False
print("请确认以下信息")
print(f"输入文件夹: {input_folder_path}")
print(f"输出文件夹: {output_folder_path}")
print(f"抽取步长: {step}")
print(f"缩放尺寸 : {resize}")

In [ ]:


if os.path.exists(output_folder_path):
    shutil.rmtree(output_folder_path)
os.makedirs(output_folder_path)


# 获取文件夹内的所有文件列表
file_list = os.listdir(input_folder_path)

# 按照文件名排序
sorted_file_list = sorted(file_list)


sorted_file_list = sorted_file_list[::step]
# 重命名文件为序号
for index, file_name in tqdm(enumerate(sorted_file_list)):

    file_path = os.path.join(input_folder_path, file_name)

        # 打开图像文件
    image = Image.open(file_path)

    # 获取图像的宽度和高度
    width, height = image.size

    # 计算裁剪的区域尺寸
    crop_height = int(height * 0.1)  # 上面 10% 的高度

    # 裁剪图像
    cropped_image = image.crop((0, crop_height, width, height))


    if resize:
        ratio = width / 1920

        resized_image = cropped_image.resize((int(width / ratio), int(height / ratio)))
    else:
        resized_image = cropped_image

    new_file_name = f"{(index+1):05d}.jpg"  # 根据需要修改文件扩展名
    new_file_path = os.path.join(output_folder_path, new_file_name)
    # shutil.move(file_path, new_file_path)
    resized_image.save(new_file_path)

### STEP2 COLMAP

In [ ]:
from utils.system_utils import run_command, run_colmap_feature_extraction, run_colmap_matching_block, run_colmap_bundle

In [ ]:
args = config.colmap_args
print(args)

len_files = len(os.listdir(os.path.join(args.source_path, "input")))
print(f"共有{len_files}个文件")

In [ ]:
colmap_command = '"{}"'.format(args.colmap_executable) if len(args.colmap_executable) > 0 else "colmap"
print(colmap_command)
magick_command = '"{}"'.format(args.magick_executable) if len(args.magick_executable) > 0 else "magick"
print(magick_command)
use_gpu = 1 if not args.no_gpu else 0

#### matching

In [ ]:
if not args.skip_matching:
    print("plase continue")
else:
    print("please go to next part")

In [ ]:
if not args.skip_matching:
    os.makedirs(args.source_path + "/distorted/sparse", exist_ok=True)

In [ ]:
## Feature extraction
if not args.skip_matching:
    feat_extracton_cmd = colmap_command + " feature_extractor " \
                                          "--database_path " + args.source_path + "/distorted/database.db \
            --image_path " + args.source_path + "/input \
            --ImageReader.single_camera 1 \
            --ImageReader.camera_model " + args.camera + " \
            --SiftExtraction.use_gpu " + str(use_gpu)
    print("start feat extraction cmd")
    print(feat_extracton_cmd)
    run_colmap_feature_extraction(feat_extracton_cmd)
    # exit_code = os.system(feat_extracton_cmd)
    # if exit_code != 0:
    #    logging.error(f"Feature extraction failed with code {exit_code}. Exiting.")
    #    exit(exit_code)

In [ ]:
## Feature matching
if not args.skip_matching:
    feat_matching_cmd = colmap_command + " exhaustive_matcher \
            --database_path " + args.source_path + "/distorted/database.db \
            --SiftMatching.use_gpu " + str(use_gpu)
    exit_code = run_colmap_matching_block(feat_matching_cmd)
    if exit_code != 0:
        print(f"Feature matching failed with code {exit_code}.")


In [ ]:


### Bundle adjustment
if not args.skip_matching:
    # The default Mapper tolerance is unnecessarily large,
    # decreasing it speeds up bundle adjustment steps.
    mapper_cmd = (colmap_command + " mapper \
            --database_path " + args.source_path + "/distorted/database.db \
            --image_path " + args.source_path + "/input \
            --output_path " + args.source_path + "/distorted/sparse \
            --Mapper.ba_global_function_tolerance=0.000001")
    exit_code = run_colmap_bundle(mapper_cmd, len_files)
    if exit_code:
        print(f"Mapper failed with code {exit_code}.")
    print("done.")


#### Image undistortion

In [ ]:
## We need to undistort our images into ideal pinhole intrinsics.
img_undist_cmd = (colmap_command + " image_undistorter \
        --image_path " + args.source_path + "/input \
        --input_path " + args.source_path + "/distorted/sparse/0 \
        --output_path " + args.source_path + "\
        --output_type COLMAP")
exit_code = run_command(img_undist_cmd)
if exit_code:
    print(f"Mapper failed with code {exit_code}")



In [ ]:

import shutil
import logging
files = os.listdir(args.source_path + "/sparse")
os.makedirs(args.source_path + "/sparse/0", exist_ok=True)
# Copy each file from the source directory to the destination directory
for file in files:
    if file == '0':
        continue
    source_file = os.path.join(args.source_path, "sparse", file)
    destination_file = os.path.join(args.source_path, "sparse", "0", file)
    shutil.move(source_file, destination_file)

if (args.resize):
    print("Copying and resizing...")

    # Resize images.
    os.makedirs(args.source_path + "/images_2", exist_ok=True)
    os.makedirs(args.source_path + "/images_4", exist_ok=True)
    os.makedirs(args.source_path + "/images_8", exist_ok=True)
    # Get the list of files in the source directory
    files = os.listdir(args.source_path + "/images")
    # Copy each file from the source directory to the destination directory
    for file in files:
        source_file = os.path.join(args.source_path, "images", file)

        destination_file = os.path.join(args.source_path, "images_2", file)
        shutil.copy2(source_file, destination_file)
        exit_code = os.system(magick_command + " mogrify -resize 50% " + destination_file)
        if exit_code != 0:
            logging.error(f"50% resize failed with code {exit_code}. Exiting.")
            exit(exit_code)

        destination_file = os.path.join(args.source_path, "images_4", file)
        shutil.copy2(source_file, destination_file)
        exit_code = os.system(magick_command + " mogrify -resize 25% " + destination_file)
        if exit_code != 0:
            logging.error(f"25% resize failed with code {exit_code}. Exiting.")
            exit(exit_code)

        destination_file = os.path.join(args.source_path, "images_8", file)
        shutil.copy2(source_file, destination_file)
        exit_code = os.system(magick_command + " mogrify -resize 12.5% " + destination_file)
        if exit_code != 0:
            logging.error(f"12.5% resize failed with code {exit_code}. Exiting.")
            exit(exit_code)

In [ ]:
print("Done.")